## Train & Track an ML Model using MLflow

- Create a Simple ML Model (e.g., Linear Regression)

In [2]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [15]:
# Generate sample data
np.random.seed(42)
X = np.random.rand(100, 1) * 10
y = 3 * X.squeeze() + np.random.randn(100) * 2 # labels with noise

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Enable MLflow tracking
mlflow.set_experiment('mlops_experiment')

with mlflow.start_run():
    model = LinearRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    # Log params & metrics
    mlflow.log_param('model_type', 'LinearRegresion')
    mlflow.log_metric('mse', mse)

    # save model
    mlflow.sklearn.log_model(model, 'model')

print(f'Model trained & logged with MSE: {mse}')

2025/04/05 00:21:43 INFO mlflow.tracking.fluent: Experiment with name 'mlops_experiment' does not exist. Creating a new experiment.
2025/04/05 00:21:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model trained & logged with MSE: 2.614798054868012


## Containerize the Model using Docker

-  Create a Dockerfile to Package the Model

## Serve the Model as an API with FastAPI (`app.py`)

In [1]:
from fastapi import FastAPI
import mlflow.pyfunc

app = FastAPI()

RUN_ID = '2648c5c0cea24050b86fcc6adaa0b565'
model = mlflow.pyfunc.load_model(f'mlruns/721578769661279971/{RUN_ID}/artifacts/model')

@app.post('/predict/')
def predict(data: dict):
    X = [[data['feature']]]
    prediction = model.predict(X)[0]
    return {'prediction': prediction}

## Run & Test the API

- Build & Run the Docker Container

In [ ]:
!docker build -t mlops_model .
!docker run -p 8000:8000 mlops_model

#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 218B 0.0s done
#1 DONE 0.0s

#2 [internal] load metadata for docker.io/library/python:3.11
#2 DONE 1.1s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/4] FROM docker.io/library/python:3.11@sha256:ebfa8696e47a68cffebb31e370a93ce57c01bc753f246ceaaef72801d1661351
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 2.64kB 0.0s done
#5 DONE 0.0s

#6 [2/4] WORKDIR /app
#6 CACHED

#7 [3/4] COPY . /app/
#7 DONE 0.0s

#8 [4/4] RUN pip install --no-cache-dir fastapi uvicorn mlflow scikit-learn
#8 2.863 Collecting fastapi
#8 3.080   Downloading fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
#8 3.209 Collecting uvicorn
#8 3.253   Downloading uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
#8 3.380 Collecting mlflow
#8 3.424   Downloading mlflow-2.21.3-py3-none-any.whl.metadata (30 kB)
#8 3.67

^C


2025/04/05 02:37:02 WARNING mlflow.utils.requirements_utils: Encountered an unexpected error (AttributeError("module 'pathlib' has no attribute '__version__'")) while detecting model dependency mismatches. Set logging level to DEBUG to see the full traceback.
/usr/local/lib/python3.11/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
INFO:     Started server process [1]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
time="2025-04-05T11:10:38+07:00" level=error msg="error waiting for container: unexpected EOF"


INFO:     172.17.0.1:52796 - "POST /predict/ HTTP/1.1" 422 Unprocessable Entity
INFO:     172.17.0.1:47238 - "POST /predict/ HTTP/1.1" 422 Unprocessable Entity
INFO:     172.17.0.1:52174 - "POST /predict/ HTTP/1.1" 422 Unprocessable Entity
INFO:     172.17.0.1:50832 - "POST /predict/ HTTP/1.1" 422 Unprocessable Entity
INFO:     172.17.0.1:37804 - "POST /predict/ HTTP/1.1" 200 OK
INFO:     172.17.0.1:53716 - "POST /predict/ HTTP/1.1" 200 OK


- Test the API

In [5]:
!curl -X POST "http://127.0.0.1:8000/predict/" -H "Content-Type: application/json" -d "{\"feature\": 5.0}"

{"prediction":14.884473953877686}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    49  100    33  100    16   2271   1101 --:--:-- --:--:-- --:--:--  3500


## Automate the Pipeline with CI/CD

- Create a Retraining Script (`train.py`)
- Set Up GitHub Actions (`.github/workflows/train.yml`)
- Commit & Push to GitHub

In [ ]:
!git init
!git add .
!git commit -m "first commit"
!git branch -M main
!git remote add origin https://github.com/HoDangCao/MLOps.git
!git push -u origin main

- How to Automate Model Deployment After Retraining?

1️⃣ Store the new model in a registry (e.g., AWS S3, Google Cloud Storage, MLflow)

2️⃣ Build & push a new Docker image with the updated model

3️⃣ Deploy to Kubernetes or a Cloud Service (AWS SageMaker, GCP Vertex AI, Azure ML)

4️⃣ Use CI/CD (GitHub Actions) to automate updates

### Option 1: Automate Model Deployment with Docker & GitHub Actions

Create `.github/workflows/deploy.yml` help:
- Runs after model retraining is complete
- Builds & pushes the new Docker image
- Deploys the updated model on a server

In [ ]:
!git add .
!git commit -m "Add automated model evaluation and kubernete files"
!git push origin main

[main b991c35] Add automated model deployment
 1 file changed, 5 insertions(+), 4 deletions(-)


To https://github.com/HoDangCao/MLOps.git
   929fa2b..b991c35  main -> main


### Option 2: Automate Model Deployment Using Kubernetes (K8s)

- Define a Kubernetes Deployment (`deployment.yaml`)
- Create a Kubernetes Service (`service.yaml`)
- Apply the Changes to Kubernetes

In [ ]:
!kubectl apply -f deployment.yaml
!kubectl apply -f service.yaml

### Option 3: Use Cloud Services (AWS/GCP/Azure)

Many cloud platforms automate model deployment.

✅ AWS SageMaker → Upload the new model to S3, deploy it with SageMaker Endpoints

✅ Google Vertex AI → Register the model in Vertex AI Model Registry, deploy with AutoML

✅ Azure ML → Use Azure ML Pipelines to redeploy automatically

### automatically retrain the model if drift is detected.

sketch a .github/workflows/evaluate.yml that:
1. Runs on a schedule ⏰
2. Checks model performance or drift 📉
3. Triggers train.yml if needed 🔁
4. Sends notifications 📬